In [1]:
%matplotlib inline
from pylab import *

In [ ]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

# Landhills Winery Case Study

# sets
set GRAPES;
set WINES;
set VARIETIES;
set VINTAGES;
set AREAS;

# grape parameters (values are in the data section)
param gVariety{GRAPES} symbolic;
param gArea{GRAPES} symbolic;
param gVintage{GRAPES} symbolic;
param gAcidity{GRAPES};
param gSugar{GRAPES};
param gAlcohol{GRAPES};
param gAvailable{GRAPES};
param gCost{GRAPES};

# wine parameters (values in the data section)
param wVariety{WINES} symbolic;
param wArea{WINES} symbolic;
param wVintage{WINES} symbolic;
param wAcidity{WINES};
param wSugar{WINES};
param wAlcohol{WINES};
param wPrice{WINES};

# decision variables
var Revenue >= 0;
var Cost >= 0;

var wProduction{WINES} >= 0, <= 500000;              # wine production in bottles
var gConsumption{GRAPES} >= 0, <= 200000;            # Grape consumption in bottles
var x{WINES,GRAPES} >= 0, <= 200000, integer;        # allocation of grapes to wines

# Objective
maximize Profit : Revenue - Cost;
s.t. CostDef: Cost == sum{g in GRAPES} gCost[g]*gConsumption[g];
s.t. RevenueDef: Revenue == sum{w in WINES} wPrice[w]*wProduction[w];
    
# Relationships (if any) among the decision variables
subject to grapes {g in GRAPES} : sum{w in WINES} x[w,g] == gConsumption[g];
subject to wines {w in WINES} : sum{g in GRAPES} x[w,g] == wProduction[w];

# Limits on Grape usage
s.t. capacity {g in GRAPES} : gConsumption[g] <= gAvailable[g];

# Varietal Content
s.t. varietal {w in WINES} : sum{g in GRAPES : wVariety[w]<> '' && wVariety[w]==gVariety[g]} x[w,g] >= 0.75*wProduction[w];

solve;

printf "Profit  = %12.2f\n", Profit;
printf "Revenue = %12.2f\n", Revenue;
printf "Cost    = %12.2f\n\n", Cost;

printf "   Wine   Production ";
printf {g in GRAPES} " %10s ", g;
printf "\n";
for {w in WINES} {
    printf "%8s ", w;
    printf "%10.1f ", wProduction[w];
    for {g in GRAPES} printf "  %10.1f", x[w,g];
    printf "\n";
}
printf "Availability         ";
for {g in GRAPES} printf " %10.1f ", gAvailable[g];


data;

set AREAS := 'SB', 'SLO';
set VARIETIES := 'CabSav', 'Merlot';
set VINTAGES := '2010', '2011';

param : GRAPES : gVariety  gArea  gVintage gAcidity, gSugar, gAlcohol,  gAvailable, gCost :=
CS_SB_2011  CabSav  SB   2011   0.35   0.12   13.5   50000   2.35 
CS_SL_2010  CabSav  SLO  2010   0.75   0.25   15.3   60000   2.60
CS_SL_2011  CabSav  SLO  2011   0.55   0.30   11.5   30000   2.10
Mr_SB_2010  Merlot  SB   2010   0.25   0.08   15.7  200000   1.55;

param : WINES  : wVariety wArea wVintage wAcidity, wSugar, wPrice :=
wSB11     'CabSav'  SB  "2011"  0.7    0.2   9.00
wSL10     CabSav  SLO "2010"  0.7    0.2   9.00
wSL11     CabSav  SLO "2011"  0.7    0.2   9.00
wCabS     CabSav  ""   ""     0.7    0.2   6.50
wMerl     Merlot  ""   ""     0.3    1.0   2.95;

end;


In [ ]:
print(open('display.txt').read())

In [ ]:
print output